In [1]:
index_file = "/home/szymon/lingwy/nkjp/nkjp_index.txt"
nkjp_path = "/home/szymon/lingwy/nkjp/pełny/"
vecs_path = "/home/szymon/lingwy/nkjp/wektory/nkjp+wiki-lemmas-all-100-skipg-ns.txt/"
skladnica_sections_index = "/home/szymon/lingwy/nkjp/skladnica_znacz/sections.txt"
skladnica_path = "/home/szymon/lingwy/nkjp/skladnica_znacz/"
pl_wordnet = "/home/szymon/lingwy/wielozn/wzięte/plwordnet_2_0_4/plwordnet_2_0_4_release/plwordnet_2_0.xml"
vecs_dim = 100
window_size = 4 # how many words to condider on both sides of the target
batch_size = window_size * 2
corp_runs = 2
learning_rate = 0.3
reg_rate = 0.005
points_per_neg_sample = 1

## Get Składnica & NKJP sentences

In [2]:
skl_ids = []
nkjp_orig_ids = []
nkjp_squeezed_ids = [] # Składnica uses "squeezed" version, w/o hyphens (-)

# This will be used later. NKJP id -> [ sent_ids ]
skl_nkjp_sent_ids = {}
# auxiliary, skl_id -> NKJP_id
nkjp_skl_ids = {}

with open(skladnica_sections_index) as index:
    for sec_name in index:
        skl_ids.append(sec_name.strip())
with open(index_file) as index:
    for fragm_id in index:
        fragm_id = fragm_id.strip()
        nkjp_orig_ids.append(fragm_id)
        nkjp_squeezed_ids.append(fragm_id.replace('-', ''))
        
bonus_skladnica_mappings = {
                            'NKJP_1M_001000-DziennikPolski1980': 'DP1980',
                            'NKJP_1M_010200-ZycieWarszawyPLUSZycie': 'ZycieWarszawy_Zycie',
                            'NKJP_1M_012750-TrybunaPLUSTrybunaLudu': 'TrybunaLudu_Trybuna',
                            'NKJP_1M_1998': 'DP1998',
                            'NKJP_1M_1999': 'DP1999',
                            'NKJP_1M_2000': 'DP2000',
                            'NKJP_1M_2001': 'DP2001',
                            'NKJP_1M_2002': 'DP2002',
                            'NKJP_1M_2003': 'DP2003',
                            'NKJP_1M_%2E': '712-3-900001'
                           }

In [3]:
# This should print Składnica ids for which we don't know their NKJP equivalents.
# (hopefully, none)
for id in skl_ids:
    if id[len('NKJP_1M_'):] in nkjp_squeezed_ids:
        sqzd_nkjp_id = id[len('NKJP_1M_'):]
        orig_nkjp_id = nkjp_orig_ids[nkjp_squeezed_ids.index(sqzd_nkjp_id)]
        skl_nkjp_sent_ids[orig_nkjp_id] = []
        nkjp_skl_ids[id] = orig_nkjp_id
    elif id[len('NKJP_1M_4scal-'):] in nkjp_squeezed_ids:
        sqzd_nkjp_id = id[len('NKJP_1M_4scal-'):]
        orig_nkjp_id = nkjp_orig_ids[nkjp_squeezed_ids.index(sqzd_nkjp_id)]
        skl_nkjp_sent_ids[orig_nkjp_id] = []
        nkjp_skl_ids[id] = orig_nkjp_id
    elif id[len('NKJP_1M_######-'):] in nkjp_squeezed_ids:
        sqzd_nkjp_id = id[len('NKJP_1M_######-'):]
        orig_nkjp_id = nkjp_orig_ids[nkjp_squeezed_ids.index(sqzd_nkjp_id)]
        skl_nkjp_sent_ids[orig_nkjp_id] = []
        nkjp_skl_ids[id] = orig_nkjp_id
    elif id in bonus_skladnica_mappings:
        orig_nkjp_id = bonus_skladnica_mappings[id]
        skl_nkjp_sent_ids[orig_nkjp_id] = []
        nkjp_skl_ids[id] = orig_nkjp_id
    else:
        print(id)

In [4]:
from lxml import etree
import os

In [5]:
# Get all the sents present in Składnica.
from operator import itemgetter

# NOTE this also contains empty lists where the sents were actually not annotated.
skl_sents = [] # pairs: word lemma, lexical unit id (or None)
skl_words = set() # all unique ones that are present
skl_synsets = set()

for skl_id in skl_ids:
    section_path = skladnica_path + skl_id
    # Each section has 1+ dirs of XML files.
    for dirname, _, __ in os.walk(section_path):
        if dirname != section_path: # ignore the dir itself
            # Get the XML files (each contains a sentence)
            for _, __, filenames in os.walk(dirname):
                for filename in filenames:
                    tree = etree.parse(dirname+'/'+filename)
                    # Check if the file has wordnet annotations.
                    if tree.find('.//plwn_interpretation') is None:
                        continue
                    
                    sent_id = filename[:-len('.xml')]
                    skl_nkjp_sent_ids[nkjp_skl_ids[skl_id]].append(sent_id)
                    
                    ###skl_nkjp_sent_ids[nkjp_skl_ids[skl_id]].append(tree.find(".//text"))
                    # Collect the list of sentence lemmas.
                    sent = []
                    for elem in tree.iterfind('node[@chosen]'):
                        if elem.attrib['chosen'] == 'true' and elem.find("terminal") is not None:
                            lexical_id = None
                            if elem.find('.//luident') is not None: # if word-sense annotation is available
                                lexical_id = elem.find('.//luident').text

                            lemma = elem.find("terminal").find("base").text
                            if lemma is None: # happens for numbers
                                lemma = elem.find("terminal").find("orth").text
                            lemma = lemma.lower()
                            sent.append((int(elem.attrib['from']), lemma, lexical_id))
                            skl_words.add(lemma)
                    sent.sort(key=itemgetter(0))
                    sent = [(token, lexical_id) for num, token, lexical_id in sent]
                    skl_sents.append(sent)

In [6]:
skl_sents[:6]

[[('bo', None),
  ('gdy', None),
  ('zostać', '74824'),
  ('proboszcz', '6516'),
  (',', None),
  ('nie', None),
  ('pozwolić', '6362'),
  ('rada', '7249'),
  ('parafialny', '376389'),
  ('wprowadzić', '80608'),
  ('się', None),
  ('procesja', '26994'),
  ('do', None),
  ('plebania', '28271'),
  ('i', None),
  ('kościół', '54419'),
  ('.', None)],
 [('-', None),
  ('taki', None),
  ('być', '81782'),
  ('u', None),
  ('my', '5224'),
  ('zwyczaj', '11870'),
  ('od', None),
  ('pokolenie', '20791'),
  ('.', None)],
 [('po', None),
  ('awantura', '143207'),
  ('z', None),
  ('rodzina', '48204'),
  ('k', None),
  ('.', None),
  ('wygrzebać', '76666'),
  ('przeciwko', None),
  ('on', None),
  ('stary', '22649'),
  ('sprawa', '8290'),
  ('.', None)],
 [('osiągnąć', '73695'),
  ('korzyść', '2839'),
  ('prestiżowy', '357856'),
  ('i', None),
  ('finansowy', '1760'),
  ('.', None)],
 [('twój', '9156'),
  ('inicjatywa', '2214'),
  (',', None),
  ('konkretny', '2764'),
  (',', None),
  ('praktyczn

In [7]:
# Read the NKJP fragments, as lists of lemmas
unique_words = set()
words_count = 0
train_sents = []
rejected_sent_ids = []

with open(index_file) as index:
    for fragm_id in index:
        fragm_id = fragm_id.strip()
        # tag is namespaced, .// for finding anywhere in the tree
        tokens_filepath = nkjp_path+fragm_id+'/ann_morphosyntax.xml'
        if not os.path.isfile(tokens_filepath):
            print('Note: cannot access {}'.format(fragm_id.strip()))
            continue
        tree = etree.parse(tokens_filepath)
        for sent_subtree in tree.iterfind('.//{http://www.tei-c.org/ns/1.0}s[@{http://www.w3.org/XML/1998/namespace}id]'):
            
            # Skip sentences analyzed in Składnica.
            sent_id = sent_subtree.get('{http://www.w3.org/XML/1998/namespace}id').replace(' ', '_')
            if fragm_id in skl_nkjp_sent_ids and sent_id in skl_nkjp_sent_ids[fragm_id]:
                rejected_sent_ids.append(fragm_id+':'+sent_id)
                continue
                
            sent_lemmas = []
            for seg in sent_subtree.iterfind('.//{http://www.tei-c.org/ns/1.0}seg'):
                form = None
                lemma = None 
                for elem in seg.iterfind('.//{http://www.tei-c.org/ns/1.0}f[@name]'):
                    if elem.attrib['name'] == 'base':
                        lemma = elem[0].text # first child <string>
                    if elem.attrib['name'] == 'orth':
                        form = elem[0].text # first child <string>
                if lemma is not None:
                    sent_lemmas.append(lemma.lower())
                else: # can happen when the form is number and there's no lemma
                    sent_lemmas.append(form.lower())
            train_sents += [[]]
            for word in sent_lemmas:
                train_sents[-1].append(word)
                words_count += 1
                unique_words.add(word)
print('{} individual sents rejected, for {} known from test corpus'.format(len(rejected_sent_ids), len(skl_sents)))

8211 individual sents rejected, for 8211 known from test corpus


In [8]:
words_count

1132084

## Get word vectors

In [9]:
import numpy as np

In [10]:
first_line = True
word_n = 0
word_to_idx = {}
idx_to_word = {}

# we'll read those from the data file
vecs_count = 0
vecs_dim = 100

# Get the vector word labels (we'll get vectors themselves in a momment).
with open(vecs_path+"data") as vecs_file:
    for line in vecs_file:
        if first_line:
            # Read metadata.
            vecs_count = int(line.split(' ')[0])
            vecs_dim = int(line.split(' ')[1])
            first_line = False
            continue
        # Read lemma base forms.
        word = line.split(' ')[0].lower()
        word_to_idx[word] = word_n
        idx_to_word[word_n] = word
        word_n += 1

In [11]:
word_vecs = np.loadtxt(vecs_path+"data", encoding="utf-8",
                       dtype=np.float32, # tensorflow's requirement
                       skiprows=1, usecols=tuple(range(1, vecs_dim+1)))

In [12]:
# Add the dummy boundary/unknown marker.
word_vecs = np.vstack([word_vecs, np.zeros((1,vecs_dim), dtype=np.float32)])
vecs_count += 1

In [13]:
# Get the word's vector, or the dummy marker.
def word_id(word):
    return word_to_idx[word] if word in word_to_idx else vecs_count-1

In [14]:
vecs_count, vecs_dim

(1549323, 100)

## Keras - training the "language/gibberish discriminator"

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, Embedding
from keras.optimizers import SGD

Using TensorFlow backend.
/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [16]:
#
# Training corpus preparation.
#

from random import randint
from math import floor

# We need a special token for cases when the target word is near the start or end of sentence.
bound_token_id = vecs_count - 1 # the zero additional vector

sample_n = 0

train = np.zeros(((words_count + words_count // points_per_neg_sample) * corp_runs,
                  window_size * 2 + 1), dtype='int')
labels = np.ones(((words_count + words_count // points_per_neg_sample) * corp_runs,),
                dtype='int')

for run_n in range(corp_runs):
    sent_n = 0
    word_n = 0
        
    while sent_n < len(train_sents) and sample_n < train.shape[0]:
        # The positive sample.
        train[sample_n, window_size] = word_id(train_sents[sent_n][word_n]) # the target word
        for j in range(window_size): # its neighbors in the window_size
            train[sample_n, j] = (word_id(train_sents[sent_n][word_n-j-1]) if word_n-j-1 >= 0
                                  else bound_token_id)
            train[sample_n, window_size+j+1] = (word_id(train_sents[sent_n][word_n+j+1])
                                                if word_n+j+1 < len(train_sents[sent_n])
                                                else bound_token_id)
        # (the "one" label is the default)
        # labels[sample_n] = 1.0
        
        # The negative sample.
        sample_n += 1
        train[sample_n,] = np.random.permutation(train[sample_n-1,])
        # (replace two random words)
        train[sample_n, randint(0, window_size*2)] = randint(0, vecs_count-1)
        train[sample_n, randint(0, window_size*2)] = randint(0, vecs_count-1)
        labels[sample_n] = 0.0
                
        sample_n += 1
        word_n += 1
        # If words are exhausted, scan the sents for one that has some words.
        try:
            while word_n == len(train_sents[sent_n]):
                word_n = 0
                sent_n += 1
        # At the end of the corpus (== exhausted sents), break the loop.
        except IndexError:
            break

In [18]:
model = Sequential()                                                                                               
model.add(Embedding(vecs_count,
                    vecs_dim,
                    weights=[word_vecs],
                    input_length=window_size * 2 + 1,
                    trainable=False))                                                                              
model.add(LSTM(96))
model.add(Dense(1))
model.add(Activation('sigmoid'))

opt = SGD(lr=learning_rate, decay=reg_rate)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [19]:
model.fit(train, labels)

Epoch 1/1
4528336/4528336 [==============================] - 1226s 271us/step - loss: 0.2458 - acc: 0.9049


In [20]:
model

In [21]:
# cleanup to save some memory
train, labels = None, None

## Loading Polish Wordnet 2 information

In [ ]:
# Relations:
# only hyper/hyponymy, similarity, synonymy
rels1 = { '60', '11', '30' }
# should be rels1 + antonymy and conversion
rels2 = { '12', '101', '102', '104', '13' }
# should be rels1 + various meronymies and meronymy-like relations
rels3 = {'14', '15' # mero/holonymy
         '20', '21', '22', '23', '24', # meronymy types
         '64', '65', '80', # taxonomic type, causation
         '106', # type
         # verb meronymies
         '112', '113', '114', '115', '119', '120', '121', '122', '128', '129',
         '130', '131', '132', '134', '136', '137', '138', '140',
         # adjective relations
         '147', '150', '151', '152' }
# all relations
# rels4 is rels2 plus rels3

In [ ]:
wordnet_xml = etree.parse(pl_wordnet)

In [ ]:
from collections import defaultdict

In [ ]:
def reverse_list_dict(dic):
    new_dic = defaultdict(list)
    for key, val in dic.items():
        for v in val:
            new_dic[v].append(key)
    return new_dic

In [ ]:
# NOTE. Theoretically, we have lex_unit ids and synsets given by annotation, but we want
# to take into account all possibilities given by raw word forms. Our model will try to
# select the correct lex_unit ids.
# NOTE2. We store the bulk of relations in foo1 dicts, foo2 and foo3 store only ones specific to them

skl_word_wordids = defaultdict(list) # word -> ids as list
skl_wordid_synsets = defaultdict(list) # to harness synset relations

skl_neighbor_syns1 = defaultdict(list) # just for collecting them, synset ids
skl_neighbor_syns2 = defaultdict(list)
skl_neighbor_syns3 = defaultdict(list)
skl_wordid_neighbor_ids1 = defaultdict(list) # just for collecting them
skl_wordid_neighbor_ids2 = defaultdict(list)
skl_wordid_neighbor_ids3 = defaultdict(list)
skl_wordid_neighbors1 = defaultdict(list) # lemmas
skl_wordid_neighbors2 = defaultdict(list)
skl_wordid_neighbors3 = defaultdict(list)

# Wordnet lexical ID's of lemmas present in Składnica.
for lex_unit in wordnet_xml.iterfind('lexical-unit'):
    if lex_unit.get('name') in skl_words:
        skl_word_wordids[lex_unit.get('name')].append(lex_unit.get('id'))

In [ ]:
# Compare words where we have their ids with the whole count
len(skl_word_wordids), len(skl_words)

In [ ]:
# words missing in wordnet
[w for w in skl_words if not w in skl_word_wordids][:100]

In [ ]:
# (reverse dict)
skl_wordid_words = reverse_list_dict(skl_word_wordids) # ids -> words

# Get the synsets.
for synset in wordnet_xml.iterfind('synset'):
    for unit in synset.iterfind('unit-id'):
        if unit.text in skl_wordid_words:
            skl_wordid_synsets[unit.text].append(synset.get('id')) # assign synset id to the wordid

In [ ]:
# Distinct wordids (more than word forms) with known synsets
len(skl_wordid_synsets)

In [ ]:
# Get lexical relations.
for lexrel in wordnet_xml.iterfind('lexicalrelations'):
    if lexrel.get('parent') in skl_wordid_words:
        if lexrel.get('relation') in rels1:
            skl_wordid_neighbor_ids1[lexrel.get('parent')].append(lexrel.get('child'))
        if lexrel.get('relation') in rels2:
            skl_wordid_neighbor_ids2[lexrel.get('parent')].append(lexrel.get('child'))
        if lexrel.get('relation') in rels3:
            skl_wordid_neighbor_ids3[lexrel.get('parent')].append(lexrel.get('child'))
    if lexrel.get('child') in skl_wordid_words:
        if lexrel.get('relation') in rels1:
            skl_wordid_neighbor_ids1[lexrel.get('child')].append(lexrel.get('parent'))
        if lexrel.get('relation') in rels2:
            skl_wordid_neighbor_ids2[lexrel.get('child')].append(lexrel.get('parent'))
        if lexrel.get('relation') in rels3:
            skl_wordid_neighbor_ids3[lexrel.get('child')].append(lexrel.get('parent'))

In [ ]:
# Wordids with purely lexical relations
len(skl_wordid_neighbor_ids1), len(skl_wordid_neighbor_ids2), len(skl_wordid_neighbor_ids3)

In [ ]:
# (reverse dict)
skl_synset_wordids = reverse_list_dict(skl_wordid_synsets)

In [ ]:
len(skl_synset_wordids)

In [ ]:
# Get synset relations.
for synrel in wordnet_xml.iterfind('synsetrelations'):
    if synrel.get('parent') in skl_synset_wordids:
        if synrel.get('relation') in rels1:
            skl_neighbor_syns1[synrel.get('child')].append(synrel.get('parent'))
        if synrel.get('relation') in rels2:
            skl_neighbor_syns2[synrel.get('child')].append(synrel.get('parent'))
        if synrel.get('relation') in rels3:
            skl_neighbor_syns3[synrel.get('child')].append(synrel.get('parent'))
    if synrel.get('child') in skl_synset_wordids:
        if synrel.get('relation') in rels1:
            skl_neighbor_syns1[synrel.get('parent')].append(synrel.get('child'))
        if synrel.get('relation') in rels2:
            skl_neighbor_syns2[synrel.get('parent')].append(synrel.get('child'))
        if synrel.get('relation') in rels3:
            skl_neighbor_syns3[synrel.get('parent')].append(synrel.get('child'))

In [ ]:
# Synsets with known relations
len(skl_neighbor_syns1), len(skl_neighbor_syns2), len(skl_neighbor_syns3)

In [ ]:
# Get additional neighbor wordids from neighbor synsets.
for synset in wordnet_xml.iterfind('synset'):
    if synset.get('id') in skl_neighbor_syns1:
        for unit in synset.iterfind('unit-id'):
            for target_syns in skl_neighbor_syns1[synset.get('id')]:
                for receiver in skl_synset_wordids[target_syns]:
                    skl_wordid_neighbor_ids1[receiver].append(unit.text)
    if synset.get('id') in skl_neighbor_syns2:
        for unit in synset.iterfind('unit-id'):
            for target_syns in skl_neighbor_syns2[synset.get('id')]:
                for receiver in skl_synset_wordids[target_syns]:
                    skl_wordid_neighbor_ids2[receiver].append(unit.text)
    if synset.get('id') in skl_neighbor_syns1:
        for unit in synset.iterfind('unit-id'):
            for target_syns in skl_neighbor_syns3[synset.get('id')]:
                for receiver in skl_synset_wordids[target_syns]:
                    skl_wordid_neighbor_ids3[receiver].append(unit.text)

In [ ]:
# How many wordids have neighbors in total
len(skl_wordid_neighbor_ids1), len(skl_wordid_neighbor_ids2), len(skl_wordid_neighbor_ids3)

In [ ]:
# (get reverses of these)
skl_neighbor_id_wordids1 = reverse_list_dict(skl_wordid_neighbor_ids1)
skl_neighbor_id_wordids2 = reverse_list_dict(skl_wordid_neighbor_ids2)
skl_neighbor_id_wordids3 = reverse_list_dict(skl_wordid_neighbor_ids3)

In [ ]:
# How many *neighbors* we know
len(skl_neighbor_id_wordids1), len(skl_neighbor_id_wordids2), len(skl_neighbor_id_wordids3)

In [ ]:
# free some memory.
skl_neighbor_syns_wordids1 = None
skl_neighbor_syns_wordids2 = None
skl_neighbor_syns_wordids3 = None
skl_synset_wordids = None

In [ ]:
# Finally, harness neighbors by their known wordids.
for lex_unit in wordnet_xml.iterfind('lexical-unit'):
    if lex_unit.get('id') in skl_neighbor_id_wordids1:
        for receiver in skl_neighbor_id_wordids1[lex_unit.get('id')]: # words "interested" in this neighbor
            skl_wordid_neighbors1[receiver].append(lex_unit.get('name'))
    if lex_unit.get('id') in skl_neighbor_id_wordids2:
        for receiver in skl_neighbor_id_wordids2[lex_unit.get('id')]:
            skl_wordid_neighbors2[receiver].append(lex_unit.get('name'))
    if lex_unit.get('id') in skl_neighbor_id_wordids3:
        for receiver in skl_neighbor_id_wordids3[lex_unit.get('id')]:
            skl_wordid_neighbors3[receiver].append(lex_unit.get('name'))

In [ ]:
for word in skl_words:
    if word in skl_word_wordids:
        for (n, wid) in enumerate(skl_word_wordids[word]):
            print(word, wid, skl_wordid_neighbors1[wid])

## Tests

In [64]:
# numbers indicate the relation group,
# a - words where at least one sense has known neighbors,
# b - words where all senses have known neighbors,
# uq - no word repetitions
a_words_checked = []
b1_words_checked = []
b2_words_checked = []
b3_words_checked = []
b4_words_checked = []
num_a = 0
num_b = 0
# also, len(words_checked)
good1a = 0.0
good1b = 0.0
#good1a_uq = 0.0
#good1b_uq = 0.0
good2a = 0.0
good2b = 0.0
#good2a_uq = 0.0
#good2b_uq = 0.0
good3a = 0.0
good3b = 0.0
#good3a_uq = 0.0
#good3b_uq = 0.0
good4a = 0.0
good4b = 0.0
#good4a_uq = 0.0
#good4b_uq = 0.0

In [65]:
sample = np.zeros((1, window_size * 2 + 1), dtype='int')
def fill_sample(lemma, sent, tid):
    sample[0, window_size] = word_id(lemma) # the target word
    for j in range(window_size): # its neighbors in the window_size
        sample[0, j] = (word_id(sent[tid-j-1])
                        if tid-j-1 >= 0
                        else bound_token_id)
        sample[0, window_size+j+1] = (word_id(sent[tid+j+1])
                                        if tid+j+1 < len(sent)
                                        else bound_token_id)

In [66]:
from random import choice

In [67]:
for sent in skl_sents:
    for (tid, token) in enumerate(sent): # these are still actually lemmas
        lemma, true_sense = token[0], token[1]
        if true_sense is None:
            continue
            
        fill_sample(lemma, sent, tid)
        reference_prob = model.predict(sample)[0][0]
        
        sense_wordids = skl_word_wordids[lemma]
        if len(sense_wordids) == 0:
            sense_wordids = skl_word_wordids[lemma + ' się']
            if len(sense_wordids) == 0:
                print(lemma, true_sense, '- no senses found, bad lemma?')
                continue
        
        # Neighbors of each sense.
        sense_ngbs1 = [skl_wordid_neighbors1[swi] for swi in sense_wordids]
        # Number of neighbors for each sense - important when calculating average for later relation subsets.
        sense_ngbcounts1 = [len(skl_wordid_neighbors1[swi]) for swi in sense_wordids]
        sense_probs1 = [reference_prob for x in sense_wordids]
        senses_considered1 = 0
        sense_ngbs2 = [skl_wordid_neighbors2[swi] for swi in sense_wordids]
        sense_ngbcounts2 = [len(skl_wordid_neighbors2[swi]) for swi in sense_wordids]
        sense_probs2 = [reference_prob for x in sense_wordids]
        senses_considered2 = 0
        sense_ngbs3 = [skl_wordid_neighbors3[swi] for swi in sense_wordids]
        sense_ngbcounts3 = [len(skl_wordid_neighbors3[swi]) for swi in sense_wordids]
        sense_probs3 = [reference_prob for x in sense_wordids]
        senses_considered3 = 0
        sense_probs4 = [reference_prob for x in sense_wordids]
        senses_considered4 = 0
        
        decision1, decision2, decision3, decision4 = '', '', '', ''
        decision1_made = False
        
        # 1st subset:
        if sum([len(x) for x in sense_ngbs1]) > 0: # we need a neighbor for at least one sense
            
            decision1_made = True
            
            for (sid, ngbs) in enumerate(sense_ngbs1):
                if len(ngbs) == 0:
                    continue
                senses_considered1 += 1
                for ngb_word in ngbs:
                    fill_sample(ngb_word, sent, tid)
                    sense_probs1[sid] += model.predict(sample)[0][0]
                    
                # We have average prob of any neighbor of this sense
                sense_probs1[sid] /= sense_ngbcounts1[sid]
                
            winners1 = [sid for (sid, p) in enumerate(sense_probs1) if p == max(sense_probs1)]
            #print(winners, sense_wordids)
            decision1 = sense_wordids[choice(winners1)]
            
            # If good, update good counts
            if decision1 == true_sense:
                good1a += 1.0
                if senses_considered1 == len(sense_wordids):
                    good1b += 1.0
            
            # Update word lists
            a_words_checked.append(lemma)
            if senses_considered1 == len(sense_wordids):
                b1_words_checked.append(lemma)
        
        # 2nd subset (1+2)
        if sum([len(x) for x in sense_ngbs2]) > 0: # there's a neighbor for at least one sense
            for (sid, ngbs) in enumerate(sense_ngbs2):
                if len(ngbs) == 0:
                    if sense_ngbcounts1[sid] > 1: # just carry the "1" average
                        sense_probs2[sid] = sense_probs1[sid]
                        senses_considered2 += 1
                    continue
                senses_considered2 += 1
                for ngb_word in ngbs:
                    fill_sample(ngb_word, sent, tid)
                    sense_probs2[sid] += model.predict(sample)[0][0]
                # We have average prob of any neighbor of this sense + carry the input from "1"
                # (note that it can be ignored due to zero neighbors)
                sense_probs2[sid] = (((sense_probs1[sid] * sense_ngbcounts1[sid]) + sense_probs2[sid])
                                    / (sense_ngbcounts1[sid] + sense_ngbcounts2[sid]))
                
            winners2 = [sid for (sid, p) in enumerate(sense_probs2) if p == max(sense_probs2)]
            decision2 = sense_wordids[choice(winners2)]
            
            if decision2 == true_sense:
                good2a += 1.0
                if senses_considered2 == len(sense_wordids):
                    good2b += 1.0
            if senses_considered2 == len(sense_wordids):
                b2_words_checked.append(lemma)
        # Carry the "1" decision for cases without specific "2" info.
        elif decision1_made:
            if decision1 == true_sense:
                good2a += 1.0
                if senses_considered1 == len(sense_wordids):
                    good2b += 1.0
            if senses_considered1 == len(sense_wordids):
                b2_words_checked.append(lemma)
               
        # 3rd subset (1+3)
        if sum([len(x) for x in sense_ngbs3]) > 0: # there's a neighbor for at least one sense
            for (sid, ngbs) in enumerate(sense_ngbs3):
                if len(ngbs) == 0:
                    if sense_ngbcounts1[sid] > 1: # just carry the "1" average
                        sense_probs3[sid] = sense_probs1[sid]
                        senses_considered3 += 1
                    continue
                senses_considered3 += 1
                for ngb_word in ngbs:
                    fill_sample(ngb_word, sent, tid)
                    sense_probs3[sid] += model.predict(sample)[0][0]
                # We have average prob of any neighbor of this sense + carry the input from "1"
                # (note that it can be ignored due to zero neighbors)
                sense_probs3[sid] = (((sense_probs1[sid] * sense_ngbcounts1[sid]) + sense_probs3[sid])
                                    / (sense_ngbcounts1[sid] + sense_ngbcounts3[sid]))
                
            winners3 = [sid for (sid, p) in enumerate(sense_probs3) if p == max(sense_probs3)]
            decision3 = sense_wordids[choice(winners3)]
            
            if decision3 == true_sense:
                good3a += 1.0
                if senses_considered3 == len(sense_wordids):
                    good3b += 1.0
            if senses_considered3 == len(sense_wordids):
                b3_words_checked.append(lemma)
        # Carry the "1" decision for cases without specific "3" info.
        elif decision1_made:
            if decision1 == true_sense:
                good3a += 1.0
                if senses_considered1 == len(sense_wordids):
                    good3b += 1.0
            if senses_considered1 == len(sense_wordids):
                b3_words_checked.append(lemma)
                
        # 4th subset (1+2+3)
        for sid, _ in enumerate(sense_wordids):
            if sense_ngbcounts1[sid] == 0 and sense_ngbcounts2[sid] == 0 and sense_ngbcounts3[sid] == 0:
                continue
            senses_considered4 += 1
            sense_probs4[sid] = ((sense_probs1[sid] * sense_ngbcounts1[sid]
                                 + sense_probs2[sid] * sense_ngbcounts2[sid]
                                 + sense_probs3[sid] * sense_ngbcounts3[sid])
                                / (sense_ngbcounts1[sid] + sense_ngbcounts2[sid] + sense_ngbcounts3[sid]))
        winners4 = [sid for (sid, p) in enumerate(sense_probs4) if p == max(sense_probs4)]
        decision4 = sense_wordids[choice(winners4)]
            
        if decision4 == true_sense:
            good4a += 1.0
            if senses_considered4 == len(sense_wordids):
                good4b += 1.0
        if senses_considered4 == len(sense_wordids):
            b4_words_checked.append(lemma)

my 5224 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
lizbona 361395 - no senses found, bad lemma?
europa 1674 - no senses found, bad lemma?
przyczynić 6969 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
my 5224 - no senses found, bad lemma?
chopin 101149 - no senses found, bad lemma?
olsztyn 63352 - no senses found, bad lemma?
olsztyn 63352 - no senses found, bad lemma?
olsztyn 63352 - no senses found, bad lemma?
polska 44430 - no senses found, bad lemma?
polska 44430 - no senses found, bad lemma?
pojawić 5887 - no senses found, bad lemma?
sld 373020 - no senses found, bad lemma?
polska 44430 - no senses found, bad lemma?
spodziewać 8223 - no senses found, bad lemma?
polska 44430 - no senses found, bad lemma?
europa 1674 - no senses found, bad lemma?
polska 44430 - no senses found, bad lemma?
rosja 30839 - no senses found, bad lemma?
bać 245 - no senses found, bad l

co 68022 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
dana 69890 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
my 5224 - no senses found, bad lemma?
taki 8961 - no senses found, bad lemma?
chiny 101090 - no senses found, bad lemma?
polska 44430 - no senses found, bad lemma?
ukraina 30840 - no senses found, bad lemma?
rumunia 56931 - no senses found, bad lemma?
bukareszt 361381 - no senses found, bad lemma?
stany 44421 - no senses found, bad lemma?
jugosławia 27787 - no senses found, bad lemma?
kazachstan 53898 - no senses found, bad lemma?
czeczenia 376195 - no senses found, bad lemma?
polska 44430 - no senses found, bad lemma?
sld 373020 - no senses found, bad lemma?
my 5224 - no senses found, bad lemma?
prl 44549 - no senses found, bad lemma?
polak 31017 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
starać 8402 - no senses found, bad lemma?
domyślać 1198 - no senses found, bad l

polska 44430 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
onejrologia 20200 - no senses found, bad lemma?
zażarty 10843 - no senses found, bad lemma?
wrocław 63311 - no senses found, bad lemma?
norwegia 53516 - no senses found, bad lemma?
spodziewać 8223 - no senses found, bad lemma?
kielce 108669 - no senses found, bad lemma?
śmiać 19138 - no senses found, bad lemma?
pozbyć 6325 - no senses found, bad lemma?
kamerun 374483 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
postarać 77738 - no senses found, bad lemma?
turcja 58780 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
opiekować 5107 - no senses found, bad lemma?
mała 1533 - no senses found, bad lemma?
objazdowy 366359 - no senses found, bad lemma?
siebie 5224 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
bieżący rok 0 - no senses found, bad lemma?
moskwa 368281 - no s

piąć 72125 - no senses found, bad lemma?
bać 245 - no senses found, bad lemma?
niemcy 56921 - no senses found, bad lemma?
polak 31017 - no senses found, bad lemma?
paryż 62331 - no senses found, bad lemma?
domyślać 1198 - no senses found, bad lemma?
onz 70421 - no senses found, bad lemma?
spodziewać 8223 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
czeczenia 376195 - no senses found, bad lemma?
rosjanin 24237 - no senses found, bad lemma?
rosjanin 24237 - no senses found, bad lemma?
taki 8961 - no senses found, bad lemma?
białoruś 30837 - no senses found, bad lemma?
taki 8961 - no senses found, bad lemma?
taki 8961 - no senses found, bad lemma?
chiny 101090 - no senses found, bad lemma?
szanghaj 361421 - no senses found, bad lemma?
polska 44430 - no senses found, bad lemma?
zelektryfikować 74277 - no senses found, bad lemma?
6-letni 355536 - no senses found, bad lemma?
spodziewać 8223 - no senses found, bad lemma?
lechita 44337 - no senses found, bad lemma?
amer

chrystus 376379 - no senses found, bad lemma?
izrael 53833 - no senses found, bad lemma?
ameryka 53462 - no senses found, bad lemma?
dziś 1517 - no senses found, bad lemma?
wielkopolska 63313 - no senses found, bad lemma?
polska 44430 - no senses found, bad lemma?
poznań 63315 - no senses found, bad lemma?
kraków 63300 - no senses found, bad lemma?
opole 110575 - no senses found, bad lemma?
brytania 18924 - no senses found, bad lemma?
gdynia 70813 - no senses found, bad lemma?
kielce 108669 - no senses found, bad lemma?
warszawa 58505 - no senses found, bad lemma?
spodobać 8222 - no senses found, bad lemma?
wyświetlić 5909 - no senses found, bad lemma?
polska 44430 - no senses found, bad lemma?
domagać 1187 - no senses found, bad lemma?
wrocław 63311 - no senses found, bad lemma?
giżycko 370623 - no senses found, bad lemma?
37-letni 355758 - no senses found, bad lemma?
kto 5224 - no senses found, bad lemma?
wietrzny 376358 - no senses found, bad lemma?
taki 8961 - no senses found, bad 

taki 8961 - no senses found, bad lemma?
siebie 5224 - no senses found, bad lemma?
taki 5306 - no senses found, bad lemma?
pojawić 5887 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
ornamencik 26704 - no senses found, bad lemma?
siebie 5224 - no senses found, bad lemma?
rozdziewiczyć 78638 - no senses found, bad lemma?
podobać 5784 - no senses found, bad lemma?
buszmen 38421 - no senses found, bad lemma?
buszmen 38421 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
bać 245 - no senses found, bad lemma?
wymykać 75999 - no senses found, bad lemma?
pomarszczyć 87603 - no senses found, bad lemma?
zapukać 72763 - no senses found, bad lemma?
europa 1674 - no senses found, bad lemma?
dowiedzieć 111433 - no senses found, bad lemma?
odzywać 4959 - no senses found, bad lemma?
nadgonić 6370 - no senses found, bad lemma?
pomiaukiwać 73461 - no senses found, bad lemma?
siebie 5224 - no senses found, bad lemma?
wisła 370564 - no senses found, bad lemma?
pol

polak 31017 - no senses found, bad lemma?
pojawić 5887 - no senses found, bad lemma?
śmiać 19138 - no senses found, bad lemma?
pojawiać 5884 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
natknąć 3896 - no senses found, bad lemma?
prześliznąć 86943 - no senses found, bad lemma?
wedrzeć 70452 - no senses found, bad lemma?
taki 8961 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
częstochowa 370587 - no senses found, bad lemma?
rozglądać 7503 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
przemian 11651 - no senses found, bad lemma?
zdarzać 11424 - no senses found, bad lemma?
rozejść 87590 - no senses found, bad lemma?
zjawić 11617 - no senses found, bad lemma?
taki 990 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
przyschnąć 77812 - no senses found, bad lemma?
mitrężyć 79356 - no senses found, bad lemma?
charchać 80324 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
szory 31903 - no 

roześmiać 97023 - no senses found, bad lemma?
obcałowywać 89505 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
wyobrazić 10419 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
polak 31017 - no senses found, bad lemma?
chrystus 376379 - no senses found, bad lemma?
wszyscy 5224 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
warszawa 58505 - no senses found, bad lemma?
taki 5306 - no senses found, bad lemma?
bać 245 - no senses found, bad lemma?
siebie 5224 - no senses found, bad lemma?
dowiedzieć 111433 - no senses found, bad lemma?
holandia 53566 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
wstrząśnienie 42434 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
kraków 63300 - no senses found, bad lemma?
włóczyć 72003 - no senses found, bad lemma?
kłębiasty 98723 - no senses found, bad lemma?
doktorwagen 6274 - n

atopowy 75 - no senses found, bad lemma?
dowiedzieć 111433 - no senses found, bad lemma?
wszyscy 4994 - no senses found, bad lemma?
siebie 5224 - no senses found, bad lemma?
my 5224 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
my 5224 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
rozlatywać 87578 - no senses found, bad lemma?
wysokogórski 1968 - no senses found, bad lemma?
jan 366928 - no senses found, bad lemma?
mickiewicz 375505 - no senses found, bad lemma?
starać 8402 - no senses found, bad lemma?
przyglądać 6977 - no senses found, bad lemma?
opiekować 5107 - no senses found, bad lemma?
wszyscy 44450 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
bać 245 - no senses found, bad lemma?
wyobrażać 10417 - no senses found, bad lemma?
bezsenny 379789 - no senses found, bad lemma?
starać 8402 - no senses found, bad lemma?
uporać 81688 - no senses found, bad lemma?
ty 5224 - no senses

białoruś 30837 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
kto 5224 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
piotruś 364217 - no senses found, bad lemma?
roczek 7428 - no senses found, bad lemma?
bać 245 - no senses found, bad lemma?
podobać 5784 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
ty 5224 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
białystok 361370 - no senses found, bad lemma?
grupowicz 968 - no senses found, bad lemma?
kijowy 371 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
kto 5224 - no senses found, bad lemma?
kto 5224 - no senses found, bad lemma?
1995 2626 - no senses found, bad lemma?
1995 2626 - no senses found, bad lemma?
kto 5224 - no senses found, bad lemma?
kto 5224 - no senses found, bad lemma?
co 68022 - no senses found, bad lemma?
sld 373020 - no senses found, bad lemma?
kto 5224 - no senses found, ba

In [68]:
print('Group 1 (some, all senses present)')
print(len(a_words_checked), good1a/len(a_words_checked))
print(len(b1_words_checked), good1a/len(b1_words_checked))
print('Group 2 (some, all senses present)')
print(len(a_words_checked), good2a/len(a_words_checked))
print(len(b2_words_checked), good1a/len(b2_words_checked))
print('Group 3 (some, all senses present)')
print(len(a_words_checked), good3a/len(a_words_checked))
print(len(b3_words_checked), good1a/len(b3_words_checked))
print('Group 4 (some, all senses present)')
print(len(a_words_checked), good4a/len(a_words_checked))
print(len(b4_words_checked), good1a/len(b4_words_checked))

Group 1 (some, all senses present)
41732 0.4093022141282469
36385 0.46945169712793733
Group 2 (some, all senses present)
41732 0.4216189015623502
34323 0.4976546339189465
Group 3 (some, all senses present)
41732 0.4137113006805329
31478 0.5426329499968232
Group 4 (some, all senses present)
41732 0.4299578261286303
37229 0.45880899298933625
